# Python Course - Tutorial 11

### Exercise 1 (Catch the bug)

After checking out the `tutorial11_testdebug` branch, find the failing test case in the portfolio analytics package and catch the error without explicit use of break points. Fix the test.

### Exercise 2 (Test coverage)

Create an HTML report for the test coverage of the portfolio analytics package in our GitHub repository.

### Exercise 3 (Parallel testing)

Run the tests in the portfolio analytics package in parallel with up to four workers.

### Exercise 4 (Testing Classes with State)

In real-world applications, we often need to test classes that maintain "state" (like a bank account balance or a portfolio inventory). Fixtures are incredibly useful here to ensure every test starts with a clean, predictable object.

You are provided with a `TradingAccount` class below. This class manages a cash balance and a dictionary of held stocks.

**Task:**
1. Create a **fixture** named `funded_account` that returns a `TradingAccount` instance starting with **10,000** USD in cash.
2. Write a test suite (3-4 functions) covering the following scenarios:
   * **Buying Stock:** Verify that buying a stock reduces cash and adds the ticker/quantity to the portfolio.
   * **Insufficient Funds:** Verify that trying to buy more stock than you can afford raises a `ValueError` (use `pytest.raises`).
   * **Selling Stock:** Verify that selling stock increases cash and decreases the portfolio quantity.
   * **Portfolio Value:** Verify the `total_value(current_prices)` method calculates the correct sum of cash + current market value of holdings.

In [ ]:
class TradingAccount:
    def __init__(self, initial_cash):
        self.cash = initial_cash
        self.portfolio = {}  # {ticker: quantity}

    def buy(self, ticker, quantity, price):
        cost = quantity * price
        if cost > self.cash:
            raise ValueError("Insufficient funds")
        self.cash -= cost
        self.portfolio[ticker] = self.portfolio.get(ticker, 0) + quantity

    def sell(self, ticker, quantity, price):
        if self.portfolio.get(ticker, 0) < quantity:
            raise ValueError("Not enough shares to sell")
        self.cash += quantity * price
        self.portfolio[ticker] -= quantity
        if self.portfolio[ticker] == 0:
            del self.portfolio[ticker]

    def total_value(self, current_prices):
        # current_prices is a dict {ticker: price}
        value = self.cash
        for ticker, qty in self.portfolio.items():
            price = current_prices.get(ticker, 0)
            value += price * qty
        return value

In [ ]:
# Your solution

### Exercise 5 (Complex Parametrization & Financial Logic)

Financial formulas often behave differently depending on the input parameters (e.g., a bond trades at par, premium, or discount depending on the coupon vs. yield). Testing these relationships via parametrization ensures your logic holds up across the entire curve.

We have a function `bond_price` that calculates the present value of a bond using the formula:
$$P = \sum_{t=1}^{T} \frac{C}{(1+r)^t} + \frac{F}{(1+r)^T}$$
Where $C$ is the coupon payment ($C=F*c$, with $c$ being the coupon rate), $F$ is face value, $r$ is the yield to maturity (YTM), and $T$ is maturity in years.

**Task:**
1. Write a test function `test_bond_pricing_scenarios` decorated with `@pytest.mark.parametrize`.
2. You must test at least **three distinct economic scenarios** (pass these as tuples in the parametrization):
   * **Par Bond:** If `coupon_rate` == `ytm`, Price should equal Face Value (1000).
   * **Discount Bond:** If `ytm` > `coupon_rate` (e.g., 5% vs 3%), Price should be **<** Face Value.
   * **Zero Coupon Bond:** If `coupon_rate` is 0, Price should be exactly $\frac{F}{(1+r)^T}$.
3. Within the test function, use `pytest.approx` for the price assertions to handle floating-point math.

In [ ]:
def bond_price(face_value, coupon_rate, ytm, years):
    coupon_payment = face_value * coupon_rate
    price = 0
    
    # Discount the coupon payments
    for t in range(1, years + 1):
        price += coupon_payment / ((1 + ytm) ** t)
        
    # Discount the face value
    price += face_value / ((1 + ytm) ** years)
    
    return price

In [ ]:
# Your solution

### Exercise 6 (Test Markers)

Sometimes tests are slow (e.g., Monte Carlo simulations) and you might want to exclude them during quick development cycles. In this case can use **markers** to categorize tests.

**Task:**
1. Write a dummy test function `test_monte_carlo_simulation` that simulates a slow process (you can use `import time; time.sleep(10)`).
2. Decorate this test with `@pytest.mark.slow` (you may need to register this marker in `pyproject.toml` or `pytest.ini` in a real project, but for this exercise, just applying the decorator is sufficient).
3. How would you run *only* the tests marked as slow using the command line?

In [ ]:
# Your solution

### Exercise 7: Initializing a new project with `uv` 

Set everything to start a **new Python project** using `uv`, moving beyond the classic `venv` + `pip` workflow used so far. Create a fresh project directory and initialize it with `uv`, targeting **Python 3.12**.

Configure the project with a small set of runtime dependencies suitable for a lightweight analysis or research toolkit (for example, `pandas`, `matplotlib`, `requests`, plus one additional package of your choice). In addition, define a separate development dependency group including tools such as `pytest`, `ruff`, and `jupyter`.
